In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

from sklearn.linear_model import LinearRegression #For regression model
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv


In [2]:
def seperate_monthly(df):
    df[['month','day','year']]=df.startdate.str.split('/',expand=True)
    df=df.drop('day', axis=1)
    df=df.drop('startdate',axis=1)
    return df

In [3]:
def seperate_y(df):
    y_df=df['contest-tmp2m-14d__tmp2m']
    X_df=df.drop('contest-tmp2m-14d__tmp2m',axis=1)
    return y_df,X_df

In [4]:
def normanlize_data(df,train_df):
    for column in df.columns:
        df[column] = df[column]  /  train_df[column].abs().max()
    return df

In [5]:
train_df=pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
train_df=train_df.dropna(axis=0)
test_df=pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
test_df=test_df.dropna(axis=0)
train_df

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
0,0,0.0,0.833333,9/1/14,237.00,29.02,31.64,29.57,30.73,29.71,...,-27.68,-37.21,8.32,9.56,-2.03,48.13,28.09,-13.50,11.90,4.58
1,1,0.0,0.833333,9/2/14,228.90,29.02,31.64,29.57,30.73,29.71,...,-21.13,-36.57,8.77,21.17,4.44,48.60,27.41,-23.77,15.44,3.42
2,2,0.0,0.833333,9/3/14,220.69,29.02,31.64,29.57,30.73,29.71,...,-10.72,-34.16,6.99,32.16,5.01,48.53,19.21,-33.16,15.11,4.82
3,3,0.0,0.833333,9/4/14,225.28,29.02,31.64,29.57,30.73,29.71,...,0.33,-31.04,6.17,39.66,-1.41,50.59,8.29,-37.22,18.24,9.74
4,4,0.0,0.833333,9/5/14,237.24,29.02,31.64,29.57,30.73,29.71,...,9.83,-31.80,7.47,38.62,-5.21,54.73,-2.58,-42.30,21.91,10.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375729,375729,1.0,0.866667,8/27/16,312.05,23.13,27.20,20.25,24.43,18.35,...,-15.64,-75.68,-3.09,6.93,-16.69,16.98,-13.85,50.25,-31.33,0.77
375730,375730,1.0,0.866667,8/28/16,305.82,23.13,27.20,20.25,24.43,18.35,...,-7.59,-76.42,-13.55,13.36,-15.96,20.45,-16.36,51.65,-30.73,10.10
375731,375731,1.0,0.866667,8/29/16,311.62,23.13,27.20,20.25,24.43,18.35,...,-6.25,-70.65,-23.93,22.62,-16.71,20.28,-15.48,48.58,-18.74,9.28
375732,375732,1.0,0.866667,8/30/16,304.54,23.13,27.20,20.25,24.43,18.35,...,-7.16,-57.67,-33.55,32.06,-16.07,16.60,-20.61,39.23,-16.26,-0.22


In [6]:
train_df=train_df[3000:]
test_df[3000:]

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
3000,378734,0.227273,0.900000,11/12/22,191.94,32.85,31.49,32.10,31.68,28.57,...,9.96,-64.54,-2.12,51.49,-39.34,16.57,-8.59,-7.95,-10.98,21.66
3001,378735,0.227273,0.900000,11/13/22,172.42,32.85,31.49,32.10,31.68,28.57,...,22.02,-73.94,-2.54,49.66,-39.14,17.09,5.20,-8.57,-13.28,32.41
3002,378736,0.227273,0.900000,11/14/22,159.51,32.85,31.49,32.10,31.68,28.57,...,31.51,-76.45,-3.79,52.06,-43.83,13.18,11.57,-12.53,-7.85,33.89
3003,378737,0.227273,0.900000,11/15/22,169.89,32.85,31.49,32.10,31.68,28.57,...,32.50,-75.06,-1.18,48.85,-47.98,13.65,12.31,-18.36,-1.12,35.25
3004,378738,0.227273,0.900000,11/16/22,191.31,32.85,31.49,32.10,31.68,28.57,...,26.15,-74.37,3.63,46.48,-47.56,14.24,14.42,-22.66,5.03,34.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31349,407083,1.000000,0.866667,12/27/22,62.72,4.60,8.71,6.05,10.08,6.39,...,74.96,-8.49,32.39,38.82,7.42,11.75,-23.62,-0.24,-5.94,51.23
31350,407084,1.000000,0.866667,12/28/22,73.41,4.60,8.71,6.05,10.08,6.39,...,88.57,0.83,26.23,37.64,13.01,17.84,-22.05,-3.03,1.31,51.45
31351,407085,1.000000,0.866667,12/29/22,70.00,4.60,8.71,6.05,10.08,6.39,...,99.43,10.90,21.06,36.53,14.15,23.12,-25.60,-5.88,9.32,45.32
31352,407086,1.000000,0.866667,12/30/22,79.81,4.60,8.71,6.05,10.08,6.39,...,109.39,21.37,20.42,36.05,6.38,29.00,-27.06,-1.42,16.06,31.88


In [7]:
#Shuffeling the data
train_df = train_df.sample(frac = 1)

In [8]:
le = preprocessing.LabelEncoder()

# Converting string labels into numbers.
train_df['climateregions__climateregion']=le.fit_transform(train_df['climateregions__climateregion'])
test_df['climateregions__climateregion']=le.fit_transform(test_df['climateregions__climateregion'])

In [9]:
print('training set prep')
train_df=seperate_monthly(train_df)
print('test set prep')
test_df=seperate_monthly(test_df)

training set prep
test set prep


In [10]:
train_df=train_df.astype('float')
test_df=test_df.astype('float')

In [11]:
y_train_df=train_df['contest-tmp2m-14d__tmp2m']
X_train_df=train_df.drop('contest-tmp2m-14d__tmp2m', axis=1)
type(y_train_df)
y_train_df=y_train_df.to_frame()
type(y_train_df)

pandas.core.frame.DataFrame

In [12]:
X_train_df=normanlize_data(X_train_df,train_df)
test_df=normanlize_data(test_df,train_df)

In [13]:
Regresion_model= model = LinearRegression() #Define model object

In [14]:
X_train_df

,index,lat,lon,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20,month,year
235107,0.625729,0.681818,0.866667,0.044746,0.095344,0.132667,0.120994,0.173419,0.138462,0.112907,...,-0.130943,-0.102441,-0.269590,0.044495,-0.288253,0.269754,0.691637,0.500564,1.000000,0.9375
144038,0.383352,0.500000,0.633333,0.225931,0.510255,0.520400,0.528302,0.433401,0.403636,0.446675,...,-0.031666,0.156386,-0.156225,0.142063,-0.145137,0.363263,-0.197295,0.132754,0.833333,0.8750
124342,0.330932,0.454545,0.666667,0.093567,0.322062,0.380239,0.320156,0.350335,0.319161,0.277445,...,-0.014454,0.450305,-0.078660,0.335879,-0.436689,-0.316701,-0.417740,0.353936,0.916667,0.8750
307922,0.819523,0.863636,0.500000,0.023314,-0.192905,-0.195948,-0.184786,-0.187117,-0.128951,-0.121773,...,-0.359072,0.469813,-0.736078,-0.427225,-0.142052,0.073884,0.000260,0.019287,0.166667,0.9375
319351,0.849941,0.909091,0.066667,0.362151,0.310144,0.319734,0.259958,0.349752,0.304895,0.364798,...,0.061906,-0.262533,0.471559,0.284408,0.077783,0.252694,0.564833,0.087074,0.416667,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31559,0.083993,0.227273,0.700000,0.104469,0.131652,0.268110,0.070081,0.190615,0.186853,0.175489,...,-0.102415,0.377724,-0.085422,-0.427885,-0.321771,0.239225,0.256730,-0.333070,0.083333,0.9375
261801,0.696774,0.772727,0.233333,0.020943,-0.010809,0.017485,-0.073375,0.044593,0.026014,0.054759,...,-0.572081,0.196818,-0.098648,0.003488,-0.220366,0.106978,0.044349,0.440221,1.000000,0.8750
370142,0.985120,1.000000,0.633333,0.233939,0.119734,0.242576,0.149446,0.213932,0.182378,0.152282,...,0.035280,0.395597,-0.201870,-0.044212,-0.134710,0.304643,-0.466901,0.773066,0.416667,0.9375
84087,0.223795,0.363636,0.666667,0.193606,0.639135,0.703025,0.665469,0.631011,0.549930,0.616167,...,0.452073,-0.029752,-1.000000,-0.260464,-0.041605,0.352873,-0.149174,-0.204038,0.750000,0.8750


In [15]:
model.fit(X_train_df, y_train_df)

LinearRegression()

In [16]:
predictions=model.predict(X_train_df)

In [17]:
final=pd.DataFrame(predictions,columns=['contest-tmp2m-14d__tmp2m'])

In [18]:
i=375734
final['index']=0
while i<407088:
    final['index'][i-375734]=i
    i+=1
final

,contest-tmp2m-14d__tmp2m,index
0,2.090534,375734
1,10.130111,375735
2,-1.449266,375736
3,-7.234293,375737
4,15.684795,375738
...,...,...
349599,4.882606,0
349600,1.462031,0
349601,12.695487,0
349602,17.055305,0


In [19]:
final.to_csv('/kaggle/working/regression_wetherprediction.csv',index=False)